In [3]:
import getpass
import os
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.tools import tool

In [4]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [5]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplica dos números enteros."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    """Suma dos números enteros."""
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    """Exponenciar la base a la potencia del exponente."""
    return base**exponent

In [6]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

In [9]:
from operator import itemgetter
from typing import Dict, List, Union
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

In [11]:
tools = [multiply, exponentiate, add]
llm_with_tools = llm.bind_tools(tools)

In [17]:
tool_map = {tool.name: tool for tool in tools}

In [18]:
def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls

In [19]:

respuesta_chain = chain = llm_with_tools | call_tools
respuesta_chain

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C846A5EB30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C846AFACB0>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.', 'parameters': {'type': 'object', 'properties': {'first_int': {'type': 'integer'}, 'second_int': {'type': 'integer'}}, 'required': ['first_int', 'second_int']}}}, {'type': 'function', 'function': {'name': 'exponentiate', 'description': 'exponentiate(base: int, exponent: int) -> int - Exponenciar la base a la potencia del exponente.', 'parameters': {'type': 'object', 'properties': {'base': {'type': 'integer'}, 'exponent': {'type': 'integer'}}, 'required': ['base', 'exponent']}}}, {'type': 'function', 'function': {'name': 'add',

In [20]:
respuesta =chain.invoke(
    "¿Cuanto es 23 veces 7, y ¿cuanto es cinco veces 18?, y ¿La suma de un millón más mil millones?, y ¿El cubo treinta y siete?"
)
respuesta

[{'name': 'multiply',
  'args': {'first_int': 23, 'second_int': 7},
  'id': 'call_zh0c',
  'output': 161},
 {'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 18},
  'id': 'call_5r26',
  'output': 90},
 {'name': 'add',
  'args': {'first_int': 1000000, 'second_int': 1000000000},
  'id': 'call_3rs5',
  'output': 1001000000},
 {'name': 'exponentiate',
  'args': {'base': 37, 'exponent': 3},
  'id': 'call_caxe',
  'output': 50653}]